<a href="https://colab.research.google.com/github/pbelevich/virtual-try-on/blob/main/virtual_try_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Virtual Try-On
This is a non-commercial educational project that uses [FrankMocap](https://github.com/facebookresearch/frankmocap) for pose estimation, [MultiGarmentNetwork](https://github.com/bharat-b7/MultiGarmentNetwork) for garment re-targeting. FrankMocap's [PyTorch3D](https://pytorch3d.org/) renderer was modified to support textures from MultiGarmentNetwork's garments.

[Sample video](https://www.shutterstock.com/video/clip-15249886-talanted-young-dancer-man-dancing-breakdance-on) was taken from [shutterstock.com](https://www.shutterstock.com/)

In [1]:
rm -rf {,.[!.],..?}*

In [2]:
!git clone --quiet -b virtual_try_on_demo https://github.com/pbelevich/frankmocap.git frankmocap

In [3]:
!git clone --quiet -b delete_absolute_paths https://github.com/pbelevich/MultiGarmentNetwork.git MultiGarmentNetwork

In [4]:
!mv frankmocap/* .

In [5]:
!mv MultiGarmentNetwork/* .

In [6]:
!rm -rf frankmocap MultiGarmentNetwork

Install dependencies

In [7]:
!apt-get install -qq libboost-dev xvfb libosmesa6-dev

Selecting previously unselected package xvfb.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.9_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.9) ...
Selecting previously unselected package libosmesa6:amd64.
Preparing to unpack .../libosmesa6_20.0.8-0ubuntu1~18.04.1_amd64.deb ...
Unpacking libosmesa6:amd64 (20.0.8-0ubuntu1~18.04.1) ...
Selecting previously unselected package libosmesa6-dev:amd64.
Preparing to unpack .../libosmesa6-dev_20.0.8-0ubuntu1~18.04.1_amd64.deb ...
Unpacking libosmesa6-dev:amd64 (20.0.8-0ubuntu1~18.04.1) ...
Setting up xvfb (2:1.19.6-1ubuntu4.9) ...
Setting up libosmesa6:amd64 (20.0.8-0ubuntu1~18.04.1) ...
Setting up libosmesa6-dev:amd64 (20.0.8-0ubuntu1~18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [8]:
!pip install -q -r docs/requirements.txt

     |████████████████████████████████| 542kB 7.1MB/s 
     |████████████████████████████████| 1.9MB 14.6MB/s 
     |████████████████████████████████| 133kB 51.5MB/s 
     |████████████████████████████████| 583kB 54.2MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 


In [9]:
!pip install -q torchgeometry chumpy opendr

     |████████████████████████████████| 51kB 4.3MB/s 


In [10]:
!test -z "$(python -c "import pytorch3d" 2>&1)" || pip install -q 'git+https://github.com/pbelevich/pytorch3d.git@stable'

     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 645kB 17.1MB/s 


In [ ]:
!sh scripts/install_pose2d.sh
!sh scripts/download_data_body_module.sh

In [12]:
!mkdir -p extra_data/smpl
!test -f smplify_public_v2.zip || gdown --id 1BwNtKvArmUdy4Cpkx1seIlsHinopdaws
!unzip -n -p mpips-smplify_public_v2.zip smplify_public/code/models/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl > extra_data/smpl/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl

Downloading...
From: https://drive.google.com/uc?id=1BwNtKvArmUdy4Cpkx1seIlsHinopdaws
To: /content/mpips-smplify_public_v2.zip
53.0MB [00:00, 111MB/s] 


In [13]:
!test -f Multi-Garmentdataset.zip || wget -q -nc https://datasets.d2.mpi-inf.mpg.de/MultiGarmentNetwork/Multi-Garmentdataset.zip

In [14]:
!test -d Multi-Garmentdataset || unzip -qn Multi-Garmentdataset.zip

In [15]:
!mkdir -p assets
!test -f mpips-smplify_public_v2.zip || gdown --id 1BwNtKvArmUdy4Cpkx1seIlsHinopdaws
!test -f assets/neutral_smpl.pkl || unzip -n -p mpips-smplify_public_v2.zip smplify_public/code/models/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl > assets/neutral_smpl.pkl

Install [mesh](https://github.com/MPI-IS/mesh)

In [16]:
!test -d mesh_dir || git clone --quiet https://github.com/pbelevich/mesh.git mesh_dir

In [ ]:
!cd mesh_dir && make all

Download sample source video

In [18]:
!mkdir -p ./sample_data
!test -f 15249886-preview.mp4 || gdown --id 1w4iQmuVP07FpHxOePkuhY2Sv3_RE8BPs -O ./sample_data/15249886-preview.mp4

Downloading...
From: https://drive.google.com/uc?id=1w4iQmuVP07FpHxOePkuhY2Sv3_RE8BPs
To: /content/sample_data/15249886-preview.mp4
100% 1.40M/1.40M [00:00<00:00, 97.3MB/s]


Run the demo

In [ ]:
!xvfb-run -a python -m demo.demo_bodymocap --input_path ./sample_data/15249886-preview.mp4 --out_dir ./mocap_output --renderer_type pytorch3d

Show the result

In [20]:
# source: https://stackoverflow.com/a/65273831

from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "./mocap_output/15249886-preview.mp4"

# Compressed video path
compressed_path = "./mocap_output/15249886-preview_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)